In [2]:
import numpy as np
import torch
import torch.nn as nn

# Загрузка данных
train_x_original = np.load('ihb.npy')  # [320, 10, 246]
train_y = np.load('train_y.npy')  # Метки (0 или 1)

# Заменяем NaN значения на 0.0
train_x_original = np.nan_to_num(train_x_original, nan=0.0)

# Преобразуем в тензоры PyTorch
train_x_tensor = torch.tensor(train_x_original, dtype=torch.float32)  # Признаки пар
train_y_tensor = torch.tensor(train_y, dtype=torch.float32)  # Метки (0 или 1)

# Создаем модель
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(246 * 2, 512)  # Входной слой
        self.fc2 = nn.Linear(512, 256)  # Второй слой
        self.fc3 = nn.Linear(256, 1)  # Выходной слой для бинарной классификации

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))  # Sigmoid для вывода в диапазоне [0, 1]
        return x

# Здесь предполагается, что модель уже обучена
model = SimpleNN()

# Функция для автоматической проверки
def automatic_test(model, data_tensor, label_tensor):
    model.eval()  # Переводим модель в режим оценки (без обновления градиентов)
    
    correct = 0
    total = 0
    
    # Проверяем все пары объектов
    for subject_1 in range(data_tensor.shape[0]):
        for subject_2 in range(subject_1 + 1, data_tensor.shape[0]):  # Проверяем только уникальные пары
            # Проверяем все временные шаги
            for timestep_1 in range(data_tensor.shape[1]):
                for timestep_2 in range(data_tensor.shape[1]):
                    # Извлекаем два набора признаков для выбранных субъектов и временных шагов
                    input_1 = data_tensor[subject_1, timestep_1, :]  # Признаки первого объекта на временном шаге
                    input_2 = data_tensor[subject_2, timestep_2, :]  # Признаки второго объекта на временном шаге
                    
                    # Объединяем их в один вектор (размер 492)
                    combined_input = torch.cat((input_1, input_2), dim=0).unsqueeze(0)  # добавляем размерность для батча
                    
                    # Реальная метка: 1, если это один и тот же человек, иначе 0
                    real_label = label_tensor[subject_1] == label_tensor[subject_2]
                    
                    # Применяем модель
                    with torch.no_grad():
                        output = model(combined_input)
                        prediction = (output > 0.5).float()  # Если предсказание > 0.5, считаем 1, иначе 0
                    
                    # Сравниваем предсказание с реальной меткой
                    correct += (prediction == real_label).sum().item()
                    total += 1  # Каждая проверка добавляет 1 к общему числу
                    
    accuracy = correct / total  # Вычисляем точность
    print(f'Точность модели: {accuracy * 100:.2f}%')
    return accuracy

# Запуск автоматической проверки:
automatic_test(model, train_x_tensor, train_y_tensor)

Точность модели: 82.10%


0.820998236677116